In [11]:
import pandas as pd
from datetime import date
from yfetch import get_stock_history

In [12]:
symbol = 'NVDA'
calendar_days = 91
trading_days = 200 #round(calendar_days * 252 / 365)

sma_days = trading_days
history = get_stock_history(symbol)
history['SMA'] = history['Close'].rolling(window=sma_days).mean()
history['SMA_dist'] = history.Close / history.SMA - 1

Fetched history for NVDA (1256 rows)


In [13]:
last_sma = history.SMA.iloc[-1]
last_close = history.Close.iloc[-1]

sma_dist = history.SMA_dist.dropna()
sma_range = (sma_dist <= sma_dist.iloc[-1]).mean()

print(symbol)
print(f'Start:    {history.index[0]}')
print(f'Price:    ${last_close:.2f} (P{sma_range:.0%})')
print(f'SMA {sma_days:3d}d: ${last_sma:.2f}')

for q in [0.90, 0.80, 0.75, 0.70, 0.60, 0.50, 0.40, 0.30, 0.25, 0.20, 0.10, 0.05]:
  dist = sma_dist.quantile(q)
  price = last_sma * (1 + dist)
  print(f'P   {q:4.0%}: ${price:6.2f} (SMA{sma_days}{dist:+.1%})')

NVDA
Start:    2020-06-29 00:00:00-04:00
Price:    $157.75 (P43%)
SMA 200d: $129.18
P    90%: $219.42 (SMA200+69.8%)
P    80%: $198.64 (SMA200+53.8%)
P    75%: $191.76 (SMA200+48.4%)
P    70%: $183.86 (SMA200+42.3%)
P    60%: $174.03 (SMA200+34.7%)
P    50%: $165.45 (SMA200+28.1%)
P    40%: $153.91 (SMA200+19.1%)
P    30%: $139.61 (SMA200+8.1%)
P    25%: $131.30 (SMA200+1.6%)
P    20%: $118.56 (SMA200-8.2%)
P    10%: $ 98.88 (SMA200-23.5%)
P     5%: $ 86.85 (SMA200-32.8%)


In [14]:
price = 200
d = price / last_sma - 1
t = (sma_dist <= d).mean()
print(f'{symbol} @ ${price:.2f} => T{trading_days}={t:.0%}')

NVDA @ $200.00 => T200=81%
